In [1]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, format_document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.document_loaders import ArxivLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 加载arXiv上的论文ReAct: Synergizing Reasoning and Acting in Language Models
loader = ArxivLoader(query = "2210.03629", load_max_docs = 1)
docs = loader.load()

# 把文本分割成200个字符为一组的片段
text_spliter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = text_spliter.split_documents(docs)

# 构建FAISS向量存储和对应的Retriever
vs = FAISS.from_documents(chunks[:10], OllamaEmbeddings(model = 'qwen2.5'))
# vs.similarity_search('What is ReAct')
retriever = vs.as_retriever()

# 构建Document转文本段落的工具函数
# （prompt中是字符串，所以需要将document转换为字符串，前4-1至4-3同理）
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template='{page_content}')
def _combine_documents(
   docs, document_prompt = DEFAULT_DOCUMENT_PROMPT, document_separator = '\n\n'
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

# 准备Model I/O三元组
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOllama(model = 'qwen2.5')

# 构建RAG链
chain = (
    {
        'context': retriever | _combine_documents,
        'question': RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke('什么是ReAct？')

'ReAct 是一种语言模型方法，它结合了推理和行动（reasoning and acting），旨在通过与简单维基百科 API 的交互来生成比基线模型更具解释性的任务解决轨迹。这种方法在 ICRL 2023 上发表，并且展示了相对于最先进的基线模型的有效性，同时提高了人类的可解释性和可信度。'